**This notebook presents code associated with Figure 3-Supplement of the following paper.**  
>Kiichi Watanabe, Hui Chiu, David J. Anderson. "Whole brain in situ mapping of neuronal activation in Drosophila during social behaviors and optogenetic stimulation" eLife (2024) (https://doi.org/10.7554/eLife.92380)  
  
**The data**  
The data set (.csv) contains pooled fluorescent intensity data from multiple individual flies.
The jGCaMP7b signal was recorded from either 0273_GAL4 or OK107-GAL4 positive neurons with optogenetic activation of P1 neurons at 1Hz. ROIs were defined manually, and fluorescence intensity was measured using Fiji/ImageJ. Please also refer to the Materials and Methods section of the paper.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

**Create a DataFrame for analysis.**  
The response to 3 repetitive photo-stimulation was recorded. 
The baseline fluorescence signal (F) was calculated by averaging the jGCaMP7b signal 30 sec before the 1st photo-stimulation.  

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('specify the path to .csv file', index_col=0)  # Update the path as needed

In [ ]:
# Set baseline window size for dF/F0 calculation
baseline_window = 30

**Calculate dF/F over time and draw a dF/F graph (mean±SEM).**  


In [ ]:
# Calculate mean across the first 30 rows and normalize
means = df.iloc[:baseline_window].mean()
df_F = (df - means) / means  # Calculate dF/F0

# Calculate mean and SEM across columns for each time point
mean_values = df_F.mean(axis=1)
sem_values = df_F.sem(axis=1)

# Create a plot DataFrame for easier plotting
plot_df = pd.DataFrame({'Time': df_F.index, 'Mean': mean_values, 'SEM': sem_values})

# Plot dF/F0 over time with SEM shaded area
plt.figure(figsize=(10, 6))
sns.lineplot(x='Time', y='Mean', data=plot_df)
plt.fill_between(plot_df['Time'], plot_df['Mean'] - plot_df['SEM'], plot_df['Mean'] + plot_df['SEM'], alpha=0.3)
plt.xticks(ticks=range(0, int(plot_df['Time'].max()) + baseline_window, baseline_window))  # Set x-axis ticks at intervals of 30
plt.xlabel('Time (s)')
plt.ylabel('dF/F0')
plt.title('Average dF/F with SEM')
plt.show()

**Compare average dF/F before and after Photostimulaton (for each stimulation).**  
1) calculate average dF/F (during each period) for each individual fly.  
2) then, calculate average of averages from different individual flies.  

In [ ]:
# Define time periods and calculate mean dF/F0 for each time period
time_periods = {
    'Pre': df_F.iloc[0:30],
    'PS1': df_F.iloc[30:60],
    'Int1': df_F.iloc[60:90],
    'PS2': df_F.iloc[90:120],
    'Int2': df_F.iloc[120:150],
    'PS3': df_F.iloc[150:180],
    'Post': df_F.iloc[180:210]
}

# Calculate mean dF/F0 for each time period and create a DataFrame for plotting
df_means = pd.DataFrame({period: data.mean() for period, data in time_periods.items()})
df_melt = df_means.melt(var_name='Time_Period', value_name='Mean_dF/F0')

# Box plot of average dF/F0 by time period
plt.figure(figsize=(10, 6))
sns.boxplot(x='Time_Period', y='Mean_dF/F0', data=df_melt, color='gray')
plt.xlabel('Time Periods')
plt.ylabel('Average dF/F')
plt.title('Mean dF/F0 by Time Period')
plt.show()

In [ ]:
# Calculate the mean dF/F for each neuron within each time period
time_periods = {
    'Pre': df_F.iloc[0:30].mean(axis=0),
    'PS1': df_F.iloc[30:60].mean(axis=0),
    'Int1': df_F.iloc[60:90].mean(axis=0),
    'PS2': df_F.iloc[90:120].mean(axis=0),
    'Int2': df_F.iloc[120:150].mean(axis=0),
    'PS3': df_F.iloc[150:180].mean(axis=0),
    'Post': df_F.iloc[180:210].mean(axis=0)
}

# Mann-Whitney U tests between specific time periods
def mann_whitney_test(period1_data, period2_data, period1_label, period2_label):
    result = stats.mannwhitneyu(period1_data, period2_data, method="asymptotic", alternative="less")
    print(f'{period1_label} vs {period2_label}: p-value = {result.pvalue}')

# Perform tests using mean dF/F across neurons
mann_whitney_test(time_periods['Pre'], time_periods['PS1'], 'Pre', 'PS1')
mann_whitney_test(time_periods['Int1'], time_periods['PS2'], 'Int1', 'PS2')
mann_whitney_test(time_periods['Int2'], time_periods['PS3'], 'Int2', 'PS3')